In [1]:
import pandas as pd

In [ ]:
file_path = 'Dados_TCC_2014_2024_Normalizados.xlsx'
xls = pd.ExcelFile(file_path)

In [3]:
# Carregar os dados de cada planilha em um dicionário
dataframes = {sheet: pd.read_excel(xls, sheet_name=sheet) for sheet in xls.sheet_names}

In [4]:
# Converter a coluna 'Data padronizada' para o formato de data em todos os dataframes
for df in dataframes.values():
    df['Data padronizada'] = pd.to_datetime(df['Data padronizada'], format='%m/%Y')

In [8]:
# Inicializar uma lista para armazenar as linhas do novo dataframe
new_data = []


In [11]:
# Obter todas as datas e valores da planilha IPCA NOVO
ipca_data = dataframes['IPCA NOVO'][['Data padronizada', 'Valor Padronizado']]

In [13]:
# Iterar sobre cada linha da planilha IPCA NOVO
for _, ipca_row in ipca_data.iterrows():
    date = ipca_row['Data padronizada']
    ipca_value = ipca_row['Valor Padronizado']
    
    # Criar uma linha inicial com a data e o valor do IPCA
    row = [date, ipca_value]
    
    # Função para coletar os valores dos últimos 12 meses de uma variável
    def get_last_12_months(sheet_name):
        values = []
        for i in range(12):
            month_date = date - pd.DateOffset(months=i)
            value = dataframes[sheet_name].loc[dataframes[sheet_name]['Data padronizada'] == month_date, 'Valor Padronizado']
            values.append(value.values[0] if not value.empty else None)
        return values[::-1]  # Inverte a lista para ordem do mês mais antigo para o mais recente

    # Adicionar os últimos 12 meses de valores de IPCA (excluindo o mês da data de referência)
    row.extend(get_last_12_months('IPCA NOVO')[:-1])

    # Adicionar os últimos 12 meses de valores de DOLAR
    row.extend(get_last_12_months('DOLAR'))

    # Adicionar os últimos 12 meses de valores de SELIC
    row.extend(get_last_12_months('SELIC'))

    # Adicionar os últimos 12 meses de valores de PETROLEO
    row.extend(get_last_12_months('PETROLEO'))

    # Adicionar os últimos 12 meses de valores de IPP NOVO
    row.extend(get_last_12_months('IPP NOVO'))

    # Adicionar os últimos 12 meses de valores de DESEMPREGO NOVO
    row.extend(get_last_12_months('DESEMPREGO NOVO'))

    # Adicionar a linha ao novo dataframe
    new_data.append(row)

In [16]:
# Criar um novo DataFrame com os dados coletados
columns = ['Data Referência', 'Valor IPCA'] + \
          [f'IPCA Mês {i-12}' for i in range(11)] + \
          [f'DOLAR Mês {i-11}' for i in range(12)] + \
          [f'SELIC Mês {i-11}' for i in range(12)] + \
          [f'PETROLEO Mês {i-11}' for i in range(12)] + \
          [f'IPP NOVO Mês {i-11}' for i in range(12)] + \
          [f'DESEMPREGO Mês {i-11}' for i in range(12)]

new_df = pd.DataFrame(new_data, columns=columns)


In [18]:
# Salvar o novo DataFrame em um arquivo Excel
output_file_path = 'C:\\Users\\JoãoEmileXimenesMuri\\Documents\\pessoal\\TCC\\Bases TCC\\Dados_TCC_2014_2024_Normalizados_TabUnica.xlsx'
new_df.to_excel(output_file_path, index=False)
print(f'Arquivo salvo em: {output_file_path}')

Arquivo salvo em: C:\Users\JoãoEmileXimenesMuri\Documents\pessoal\TCC\Bases TCC\Dados_TCC_2014_2024_Normalizados_TabUnica.xlsx
